In [25]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import sklearn as sk
import sklearn.datasets
from pandas import Series, DataFrame
from sklearn import tree


# Import and Preview

In [1]:
print("Gauss")

Gauss


In [ ]:
fraud = pd.read_csv('fraud_data.csv')
fraud.head()

In [ ]:
fraud.describe()

In [ ]:
fraud['type'] = fraud['type'].map({'PAYMENT': 0, 'TRANSFER': 1, 'CASH_OUT': 2})

In [15]:
X = fraud.drop(columns=['isFraud', 'nameOrig', 'nameDest'])

In [16]:
y = fraud['isFraud']

In [17]:
model = tree.DecisionTreeClassifier(max_depth=5)

In [19]:
np.where(np.isnan(X))

(array([      9,      10,      21, ..., 6362303, 6362315, 6362323]),
 array([1, 1, 1, ..., 1, 1, 1]))

In [23]:
pd.DataFrame(X).fillna(0)

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFlaggedFraud
0,1,0.0,9839.64,170136.00,160296.36,0.00,0.00,0
1,1,0.0,1864.28,21249.00,19384.72,0.00,0.00,0
2,1,1.0,181.00,181.00,0.00,0.00,0.00,0
3,1,2.0,181.00,181.00,0.00,21182.00,0.00,0
4,1,0.0,11668.14,41554.00,29885.86,0.00,0.00,0
...,...,...,...,...,...,...,...,...
6362615,743,2.0,339682.13,339682.13,0.00,0.00,339682.13,0
6362616,743,1.0,6311409.28,6311409.28,0.00,0.00,0.00,0
6362617,743,2.0,6311409.28,6311409.28,0.00,68488.84,6379898.11,0
6362618,743,1.0,850002.52,850002.52,0.00,0.00,0.00,0


In [24]:
model = model.fit(X, y)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
model

# Explore

Check for missing values in data set

In [ ]:
fraud.isna().sum()

In [ ]:
fraud.count()

No NULL values in data set

Check basic summary stats

In [ ]:
fraud.describe()

In [ ]:
pd.set_option('display.max_rows', None)
fraud.loc[(fraud['isFraud']==1) & (fraud['isFlaggedFraud']==1)]

In [ ]:
pd.set_option('display.max_rows', None)
fraud.loc[(fraud['isFlaggedFraud']==1)]

# Visualize

In [ ]:
## Imports
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
LABELS = ["Legitimate", "Fraud"]
count_classes = pd.value_counts(fraud['isFraud'], sort = True)
count_classes.plot(kind = 'bar', rot=0)
plt.title("Legitimate vs Fraud Transactions")
plt.xticks(range(2), LABELS)
# plt.xlabel("")
plt.ylabel("Frequency");


In [ ]:
# print(f"Number of transactions analyzed: {fraud[]})
print(f"Number of actual fraud cases: {fraud['isFraud'].sum()}")
print(f"Number of transactions flagged as fraud: {fraud['isFlaggedFraud'].sum()}")
print(f"only {round((fraud['isFlaggedFraud'].sum())/(fraud['isFraud'].sum()),3)*100}% of trasactions are being caught")

In [ ]:
# frauds = fraud[fraud.isFraud == 1]
# legitimate = fraud[fraud.isFraud == 0]

In [ ]:
fraud_amt = fraud.loc[(fraud['isFraud'] == 1) & 
                         (fraud['amount'])]
frauds = fraud.loc[(fraud['isFraud'] == 1)]
legit = fraud.loc[(fraud['isFraud']== 0)]

In [ ]:
fraud_amt['amount'].describe()

## Distribution of fraud amounts

In [ ]:
total_lost = round(fraud_amt['amount'].sum(),2)
total_cases = frauds['isFraud'].sum()
total_legit = fraud.step.count()-total_cases
print(f"Toal amount lost to fraud: ${total_lost}")
print(f"Total cases: {total_cases}")
print(f"total legit trans: {total_legit}")

In [ ]:
fig=plt.figure(figsize=(12,8))
plt.hist(fraud_amt['amount'],bins=25, alpha=0.3)

In [ ]:
# fig=plt.figure(figsize=(8,20))
fig1, ax1 = plt.subplots()
fig1 = plt.figure(figsize=(20,5))
ax1.set_title('Basic Plot')
ax1.boxplot(fraud['amount'])

The majority of fraud is occuring in the amounts between 0 and 2 million.  Need to examine that range.  However, the spike at the far right end indicates that while still small in frequency by comparison, The amount of money lost in that bin is significant.

#### Greater than 8mil

In [ ]:
fig=plt.figure(figsize=(12,8))
majority_amt7 = fraud_amt.loc[(fraud_amt['amount'] > 8000000)] 
plt.hist(majority_amt7['amount'])
money_lost = majority_amt7['amount'].sum()
cases1 = majority_amt7['isFraud'].sum()*100
proportion_cases = round((cases1/total_cases),2)
proportion = round((money_lost/total_lost*100),2) 
print(f"Total money lost in this range: {money_lost}")
print(f"Proportion to total cases: {proportion_cases}%")
print(f"proportion to total($): {proportion}%")

In [ ]:
fig=plt.figure(figsize=(12,8))
fig1, ax1 = plt.subplots()
ax1.set_title('Basic Plot')
ax1.boxplot(majority_amt7['amount'])

#### Less than 2mil

In [ ]:
fig=plt.figure(figsize=(12,8))
majority_amt = fraud_amt.loc[(fraud_amt['amount'] < 2000000)] 
plt.hist(majority_amt['amount'])
money_lost = majority_amt['amount'].sum()
cases2 = majority_amt['isFraud'].sum()*100
proportion_cases = round((cases2/total_cases),2)
proportion = round((money_lost/total_lost*100),2) 
print(f"Total money lost in this range: {money_lost}")
print(f"Proportion to total cases: {proportion_cases}%")
print(f"proportion to total($): {proportion}%")

#### Less than 500000

In [ ]:
fig=plt.figure(figsize=(12,8))
majority_amt2 = fraud_amt.loc[(fraud_amt['amount'] < 500000)] 
plt.hist(majority_amt2['amount'])
money_lost = majority_amt2['amount'].sum()
cases3 = majority_amt2['isFraud'].sum()*100
proportion_cases = round((cases3/total_cases),2)
proportion = round((money_lost/total_lost*100),2) 
print(f"Total money lost in this range: {money_lost}")
print(f"Proportion to total cases: {proportion_cases}%")
print(f"proportion to total($): {proportion}%")

#### Less than 200000

In [ ]:
fig=plt.figure(figsize=(12,8))
majority_amt3 = fraud_amt.loc[(fraud_amt['amount'] < 100000)] 
plt.hist(majority_amt3['amount'])
money_lost = majority_amt3['amount'].sum()
cases3 = majority_amt3['isFraud'].sum()*100
proportion_cases = round((cases3/total_cases),2)
proportion = round((money_lost/total_lost*100),2) 
print(f"Total money lost in this range: {money_lost}")
print(f"Proportion to total cases: {proportion_cases}%")
print(f"proportion to total($): {proportion}%")

#### Less than 20k

In [ ]:
# majority_amt4 = fraud_amt.loc[(fraud_amt['amount'] < 20000)] 
# plt.hist(majority_amt4['amount'])
# money_lost = majority_amt4['amount'].sum()
# proportion = round((money_lost/total_lost*100),2) 
# print(f"Total money lost in this range: {money_lost}")
# print(f"proportion to total: {proportion}%")

#### less than 5k

In [ ]:
# majority_amt5 = fraud_amt.loc[(fraud_amt['amount'] < 5000)] 
# plt.hist(majority_amt5['amount'])
# money_lost = majority_amt5['amount'].sum()
# proportion = round((money_lost/total_lost*100),2) 
# print(f"Total money lost in this range: {money_lost}")
# print(f"proportion to total: {proportion}%")

#### Less than 1k

In [ ]:
# majority_amt6 = fraud_amt.loc[(fraud_amt['amount'] < 1000)] 
# plt.hist(majority_amt6['amount'])
# money_lost = majority_amt6['amount'].sum()
# proportion = round((money_lost/total_lost*100),2) 
# print(f"Total money lost in this range: {money_lost}")
# print(f"proportion to total: {proportion}%")

## Times of day

#### TODO: Also convert for day of week, and month of year

In [ ]:
frauds = frauds.assign(time = frauds['step']%24) 

In [ ]:
frauds.head()

In [ ]:
legit = legit.assign(time = legit['step']%24)

In [ ]:
legit.head()

In [ ]:
fraud_time ={}
for i in range(0,total_cases,1):
    time = frauds.iloc[i].time
    if time not in fraud_time:
        fraud_time[time] = 1
    else:
        fraud_time[time] += 1

In [ ]:
legit_time={}        
for i in range(0,total_legit,1):
    time = legit.iloc[i].time
    if time not in legit_time:
        legit_time[time] = 1        
    else:
        legit_time[time] += 1

#### Frequency of fraud transactions at time intervals

In [ ]:
plt.bar(fraud_time.keys(), fraud_time.values())

Looks like no correlation between time of day and fraud...



#### Frequency of legit transactions at time intervals

In [ ]:
plt.bar(legit_time.keys(), legit_time.values())

...but there's definately time's of day for 'normal' business

## Confusion Matrix

In [ ]:
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.metrics import plot_confusion_matrix
import seaborn as sn

In [ ]:
# Predicted values
y_pred = fraud.isFlaggedFraud
# Actual values
y_act = fraud.isFraud
# plt.show()
print(metrics.confusion_matrix(y_act, y_pred, labels=[1, 0]))
print(metrics.classification_report(y_act, y_pred, labels=[1, 0]))
# metrics.confusion_matrix(y_act, y_pred, labels=[1, 0])
# metrics.classification_report(y_act, y_pred, labels=[1, 0])
g_conf = metrics.confusion_matrix(y_act, y_pred, labels=[1, 0])
df_g_conf = pd.DataFrame(g_conf, range(2), range(2))
sn.set(font_scale=1.4)
sn.heatmap(df_g_conf, annot=True, annot_kws={"size": 16})

plt.show()